In [22]:
# !pip install ultralytics -q

In [23]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# %cd drive/MyDrive/analysis

[Errno 2] No such file or directory: 'drive/MyDrive/analysis'
/content/drive/MyDrive/analysis


In [25]:
import os
import pandas as pd
import numpy as np
import cv2
from ultralytics import YOLO
from glob import glob

In [26]:
model_path = './keypoint_detection/analysis/weights/best.pt'
model = YOLO(model_path)

In [27]:
true_df = pd.DataFrame(columns=['path', 'phase', 'TOE_x', 'TOE_y', 'HOSEL_x', 'HOSEL_y', 'GRIP_x', 'GRIP_y'])
pred_df = pd.DataFrame(columns=['path', 'phase', 'TOE_x', 'TOE_y', 'HOSEL_x', 'HOSEL_y', 'GRIP_x', 'GRIP_y'])
conf_df = pd.DataFrame(columns=['path', 'phase', 'TOE_conf', 'HOSEL_conf', 'GRIP_conf'])

In [28]:
count = 0
for phase in ['train', 'val']:
    image_dir = os.path.join('./dataset/images', phase)
    label_dir = os.path.join('./dataset/labels', phase)

    for image_path in glob(os.path.join(image_dir, "*.jpg")):
        count += 1
        print(f"\r count: {count}", end="")
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        label_path = os.path.join(label_dir, base_name+'.txt')

        results = model(cv2.imread(image_path), verbose=False)

        boxes = results[0].cpu().numpy().boxes
        keypoints = results[0].cpu().numpy().keypoints
        target_idxs = [i for i, c in enumerate(boxes.cls)]

        if len(target_idxs) > 0:
            idx = np.argmax(boxes.conf[target_idxs])

            toe, hosel, grip = keypoints.xyn[idx]
            toe_x, toe_y = toe
            hosel_x, hosel_y = hosel
            grip_x, grip_y = grip
            toe_conf, hosel_conf, grip_conf = keypoints.conf[idx]

            pred_df.loc[len(pred_df)] = [base_name, phase, toe_x, toe_y, hosel_x, hosel_y, grip_x, grip_y]
            conf_df.loc[len(conf_df)] = [base_name, phase, toe_conf, hosel_conf, grip_conf]
        else:
            pred_df.loc[len(pred_df)] = [base_name, phase, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
            conf_df.loc[len(conf_df)] = [base_name, phase, np.nan, np.nan, np.nan]

        with open(label_path, 'r') as f:
            data = f.read().split(' ')
        true_df.loc[len(true_df)]= [base_name, phase, data[5], data[6], data[8], data[9], data[11], data[12]]

 count: 2210

In [29]:
true_df.to_csv('true_data.csv', index=False, header=True)
pred_df.to_csv('pred_data.csv', index=False, header=True)
conf_df.to_csv('conf_data.csv', index=False, header=True)